# Landcover Training Data

#### Sources: 
- https://blog.gishub.org/earth-engine-tutorial-32-machine-learning-with-earth-engine-supervised-classification
- https://geohackweek.github.io/GoogleEarthEngine/05-classify-imagery/
- https://ceholden.github.io/open-geo-tutorial/python/chapter_5_classification.html
- GEE Documentation


### Import libraries


In [3]:
import ee
import geemap
from geemap import *
import json
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
import os
# !pip install geemap


In [4]:
Map = geemap.Map()

### Organise Data and Visualise Data

In [6]:
#import studyRegion data
file_path = os.path.abspath('/Users/joycelynlongdon/Desktop/Cambridge/CambridgeCoding/MRES/Data/GeoJSONS/PIREDD_Plataue.geojson')

#define area of interest
def open_aoi_json(aoi_file_path):
    with open(aoi_file_path) as f:
        studyRegion = json.load(f)
    studyRegion = ee.FeatureCollection(studyRegion).first().geometry()
    #print(aoi)
    return studyRegion

studyRegion = open_aoi_json(file_path)


In [16]:
#process satellite imagery
point = ee.Geometry.Point(21, -4)

def cloudMask(image):
  #Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    #Get the pixel QA band.
    qa = image.select('pixel_qa')
    #Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)
#making a cloud free Landsat 8 Surface Reflectance Composite

l8_sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
l8_sr_masked = l8_sr.map(cloudMask)
l8_sr_med = l8_sr_masked.median().select(['B1', 'B3', 'B2','B4','B5','B6','B7'])
l8_image = l8_sr_med.clip(studyRegion)
#reduce image collection into one multiband image taking the median value of each pixel

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'] #True Colour Composite
}


Map.centerObject(point, 6)
Map.addLayer(l8_sr_masked, vis_params, "Landsat-8")
Map
#Map.addLayer(aoi, "Mai_Ndombe");


Map(bottom=8538.0, center=[-4, 21], controls=(WidgetControl(options=['position'], widget=HBox(children=(Toggle…

### Create training set

In [17]:
#set training region extent
region = ee.Geometry.Point([25.47, 7.19]).buffer(10000) #the town of Kabongo in the South East where there is a good range of classes

In [21]:
#import training data set and vectorise to make into a FEatureCollection which is necesary for GEE
ESA_CCI = '/Users/joycelynlongdon/Desktop/Cambridge/CambridgeCoding/MRES/Data/ESA_CCI/CCI_Landcover_Maps/ESA_CCI_LC_Map_2013.tif'
ESA_CCI = ee.Image(ESA_CCI)
ESA_CCI = ESA_CCI.reduceToVectores({
    "geometry":studyRegion,
    "crs": 'EPSG:4326',
    "scale": 300,
    "labelProperty": landcover_data
})

AttributeError: 'Image' object has no attribute 'reduceToVectores'